In [1]:
import os

os.environ["SEED"] = "0"
os.environ["TRAIN_DIR"] = "TRAIN"

import numpy as np

from src.aac.AAC import AAC
from src.constant import DATA_DIR, DATABASE_DIR, MAIN_DIR
from src.database import DB
from src.database.queries import get_model_training_data, get_solvers_count
from src.experiment import parhydra
from src.instance.TSP_Instance import TSP_from_index_file, set_n22_cut_off_time
from src.log import logger
from src.solver.Portfolio import Portfolio
from src.solver.TSP_LKH_Solver import TSP_LKH_Solver
from src.surrogate.SurrogatePolicy import (
    EvaluationSurrogatePolicyA,
    EvaluationSurrogatePolicyB,
    EvaluationSurrogatePolicyC,
    IterationSurrogatePolicyA,
    IterationSurrogatePolicyB,
    SurrogatePolicy,
    TestSurrogatePolicy,
)
from src.surrogate.wrapper import EmptyWrapper, SurvivalFunctionWrapper

In [2]:
import pickle
with open(MAIN_DIR / "archive" / "phase1" / "results" / "permutation" / "HO" / "coxph_incumbent.pkl", "rb") as f:
    coxph_incumbent = pickle.load(f)

In [3]:
train_instances = TSP_from_index_file(
    filepath=DATA_DIR / "TSP" / "TRAIN" / "index.json",
    cut_off_cost=100,
    cut_off_time=10,
    n=5,
)
test_instances = TSP_from_index_file(
    filepath=DATA_DIR / "TSP" / "TEST" / "index.json",
    cut_off_cost=1000,
    cut_off_time=100,
    n=250,
)

In [4]:
train_instances = set_n22_cut_off_time(train_instances, reference_cut_off_time=10.0)

In [5]:
# surrogate_policy = SurrogatePolicy(
#     estimator_wrapper=EmptyWrapper(),
#     first_fit_solver_count=1,
#     refit_solver_count=2,
# )

# surrogate_policy = TestSurrogatePolicy(
#     estimator_wrapper=EmptyWrapper(),
#     first_fit_solver_count=1,
#     refit_solver_count=2,
# )

# surrogate_policy = TestSurrogatePolicy(
#     estimator_wrapper=SurvivalFunctionWrapper(**coxph_incumbent),
#     first_fit_solver_count=5,
#     refit_solver_count=2,
# )

# surrogate_policy = EvaluationSurrogatePolicyA(
#     estimator_wrapper=SurvivalFunctionWrapper(**coxph_incumbent),
#     first_fit_solver_count=5,
#     refit_solver_count=2,
#     pct_chance=0.5
# )

# surrogate_policy = EvaluationSurrogatePolicyB(
#     estimator_wrapper=SurvivalFunctionWrapper(**coxph_incumbent),
#     first_fit_solver_count=5,
#     refit_solver_count=2,
#     reevaluate_pct=0.5
# )

# surrogate_policy = EvaluationSurrogatePolicyC(
#     estimator_wrapper=SurvivalFunctionWrapper(**coxph_incumbent),
#     first_fit_solver_count=5,
#     refit_solver_count=2,
#     reevaluate_factor=1.0
# )

# surrogate_policy = IterationSurrogatePolicyA(
#     estimator_wrapper=SurvivalFunctionWrapper(**coxph_incumbent),
#     first_fit_solver_count=5,
#     refit_solver_count=2,
#     iter_diff=2
# )

surrogate_policy = IterationSurrogatePolicyB(
    estimator_wrapper=SurvivalFunctionWrapper(**coxph_incumbent),
    first_fit_solver_count=5,
    refit_solver_count=2,
)

In [6]:
SOLVERS_N = 1
ATTEMPTS_N = 1
MAX_ITER = 10

portfolio = parhydra(
    train_instances=train_instances,
    surrogate_policy=surrogate_policy,
    SOLVERS_N=SOLVERS_N,
    ATTEMPTS_N=ATTEMPTS_N,
    MAX_ITER=MAX_ITER,
)

[2025-04-29 23:41:07] INFO      Solver 1/1
[2025-04-29 23:41:07] INFO      Attempt 1/1
[2025-04-29 23:41:07] DEBUG     AAC(prefix=config;solver=1;attempt=1, iter=1/10)
[2025-04-29 23:41:07] DEBUG     SurrogatePolicy(estimator_wrapper=<src.surrogate.wrapper.SurvivalFunctionWrapper.SurvivalFunctionWrapper object at 0x000001FDC7C33E80>, first_fit_solver_count=5, refit_solver_count=2, last_fit_solver_count=0, is_fitted=False)
[2025-04-29 23:41:07] DEBUG     SurrogatePolicy.notify_iter(solver_count=0)
[2025-04-29 23:41:07] DEBUG     Portfolio.evaluate(config;solver=1;attempt=1;aac_iter=1)
[2025-04-29 23:41:07] DEBUG     Portfolio(size=1)[Solver(id=52980316141715150)]
[2025-04-29 23:41:07] DEBUG     solve(prefix=config;solver=1;attempt=1;aac_iter=1, solver=Solver(id=52980316141715150), instance=TSP_Instance(filepath=TSP/TRAIN/cluster_netgen/000.tsp))
[2025-04-29 23:41:07] DEBUG     calculate_features(instance=TSP_Instance(filepath=TSP/TRAIN/cluster_netgen/000.tsp))
[2025-04-29 23:41:11] DEBU

In [9]:
DB().get_results()

,id,prefix,solver_id,instance_id,cost,time,cut_off_cost,cut_off_time,cached,surrogate,error
0,config;solver=1;attempt=1;aac_iter=1_529803161...,config;solver=1;attempt=1;aac_iter=1,52980316141715150,1578235657227061094,6.900000,4.374186,6.9,0.69,0,0,0
1,config;solver=1;attempt=1;aac_iter=1_529803161...,config;solver=1;attempt=1;aac_iter=1,52980316141715150,1099564706851039575,4.400000,1.121611,4.4,0.44,0,0,0
2,config;solver=1;attempt=1;aac_iter=1_529803161...,config;solver=1;attempt=1;aac_iter=1,52980316141715150,1630180243408159957,52.300000,9.516063,52.3,5.23,0,0,0
3,config;solver=1;attempt=1;aac_iter=1_529803161...,config;solver=1;attempt=1;aac_iter=1,52980316141715150,1877163610474807747,2.260000,3.333383,30.4,3.04,0,0,0
4,config;solver=1;attempt=1;aac_iter=1_529803161...,config;solver=1;attempt=1;aac_iter=1,52980316141715150,1170191481226940335,59.300000,10.334421,59.3,5.93,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
225,config;solver=2;attempt=1;aac_iter=1;surrogate...,config;solver=2;attempt=1;aac_iter=1;surrogate,85558984741464718,1099564706851039575,1.219047,0.000000,4.4,0.44,0,1,0
226,config;solver=2;attempt=1;aac_iter=1;surrogate...,config;solver=2;attempt=1;aac_iter=1;surrogate,1711584254718207942,1170191481226940335,0.580000,0.000000,59.3,5.93,1,0,0
227,config;solver=2;attempt=1;aac_iter=1;surrogate...,config;solver=2;attempt=1;aac_iter=1;surrogate,85558984741464718,1170191481226940335,1.647612,0.000000,59.3,5.93,0,1,0
228,config;solver=2;attempt=1;aac_iter=1;surrogate...,config;solver=2;attempt=1;aac_iter=1;surrogate,1711584254718207942,1877163610474807747,0.070000,0.000000,30.4,3.04,1,0,0
